## Install dependencies

In [ ]:
try:
    # some dependencies might not be installed (e.g. on Google Colab)
    import fmpy
except:
    # install the required packages
    !pip install fmpy
    
try:
    # determine if we're running on Google Colab
    from google.colab import files
    is_colab = True
except:
    is_colab = False

## Download files

In [ ]:
from fmpy.util import download_file

# download a file from an URL
download_file(url='https://github.com/t-sommer/fmi-cloud-tutorial/raw/main/Heater.fmu',
              checksum='fe5beced10f7dcf3c3aa453dafa6f60da0f7a5071ad8b2d650bdc7dcc6237246')

## Display text and images

Add _formatted_ **text**, images and tables using [Markdown](https://www.markdownguide.org/cheat-sheet/):

### Heater

A constant voltage of 10 V is applied to a temperature dependent resistor of 10*(1+(T-20C)/(235+20C)) Ohms, whose losses v^2/r are dissipated via a thermal conductance of 0.1 W/K to ambient temperature 20 degree C. The resistor is assumed to have a thermal capacity of 1 J/K, having ambient temperature at the beginning of the experiment.

#### Parameters

| Name  | Default | Description              |
|-------|--------:|--------------------------|
| TAmb  | 293.15  | Ambient temperature [K]  |
| TDif  | 2       | Error in temperature [K] |


![Heater model](https://raw.githubusercontent.com/t-sommer/fmi-cloud-tutorial/main/assets/Heater.png)

## Get information about the FMU

In [ ]:
from fmpy import dump

# print information about the FMU
dump('Heater.fmu', causalities=['input', 'output', 'parameter'])

## Simulate the FMU and plot the result

In [ ]:
# hint: you can conveniently generat a Jupyter notebook with the
# FMPy GUI (Tools > Create Jupyter Notebook...) or with the command
# fmpy create-jupyter-notebook <MyModel.fmu>

import os
import fmpy
from fmpy import simulate_fmu, plot_result, read_csv

fmpy.plot_library = 'plotly'  # experimental


filename = 'Heater.fmu'

start_values = {
    # variable                           start   unit      description
    'TAmb':                            (293.15, 'K'),    # Ambient temperature
    'TDif':                                 (2, 'K'),    # Error in temperature
    'constantVoltage.V':                   (10, 'V'),    # Value of constant voltage
    'heatCapacitor.C':                      (1, 'J/K'),  # Heat capacity of element (= cp*m)
    'resistor.R':                          (10, 'Ohm'),  # Resistance at temperature T_ref
    'resistor.T_ref':                  (293.15, 'K'),    # Reference temperature
    'resistor.alpha':     (0.00392156862745098, '1/K'),  # Temperature coefficient of resistance (R_actual = R*(1 + alpha*(T_heatPort - T_ref))
    'thermalConductor.G':                 (0.1, 'W/K'),  # Constant thermal conductance of material
}

# read input.csv (if it exists)
input = read_csv("input.csv") if os.path.isfile("input.csv") else None

# define the variables to record
output = [
    'u',     # true => switch open, false => p--n connected
    'TRes',  # Resulting temperature
    'T',     # Absolute temperature in degree Celsius as output signal
]

# simulate the FMU
result = simulate_fmu(filename, start_values=start_values, input=input, output=output, stop_time=50.0)

# plot the simulation results
plot_result(result)

## Upload files to the notebook

In [ ]:
if is_colab:
    # this only works in Google Colab
    uploaded = files.upload()
else:
    # use this in Jupyter
    from ipywidgets import FileUpload
    from IPython.display import display
    import time
    uploaded = FileUpload(accept='.csv')
    display(uploaded)

    

In [ ]:
if not is_colab:
    if uploaded.data:
        # write the file to disk
        with open("input.csv", "w+b") as i:
            i.write(uploaded.data[0])
        print("File saved as 'input.csv'.")
    else:
        print("No file was uploaded!")

## Download files from the notebook

In [ ]:
from fmpy import write_csv

# write the result to disk as a CSV
write_csv('Heater_out.csv', result)

if is_colab:
    # this only works in Google Colab
    files.download('Heater_out.csv')
else:
    # use this in Jupyter
    from IPython.display import FileLink
    from fmpy import write_csv
    FileLink('Heater_out.csv')

## Show docstring

In [ ]:
help(dump)

## Open documentation

In [ ]:
dump?

## Access the source code

In [ ]:
dump??

## Show progress of long running simulations

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
from time import sleep

stop_time = 100

progress = widgets.FloatProgress(value=0, min=0, max=stop_time, description='Simulating...')

# worker function that runs the simulation asynchronously
def work(progress):
    
    global result
    
    # callback function that gets called after every simulation step
    def step_finished(time, recorder):
        sleep(0.01)  # slow down the simulation
        progress.value = time
        return True
    
    result = simulate_fmu('Heater.fmu', stop_time=stop_time, step_finished=step_finished)

# show the progress bar
display(progress)

# start the simulation thread
thread = threading.Thread(target=work, args=(progress,))
thread.start()
thread.join()

# hide the progress bar
progress.layout.display = "none"

# plot the simulation result
plot_result(result)

## Move code to a library module

In [ ]:
# load the library file
download_file(url='https://github.com/t-sommer/fmi-cloud-tutorial/raw/main/heater.py',
              checksum='81de0803d9072f9df5544e32b984787510b89ee0bbec374883dc4c48f6d4ed35')

from heater import simulate_heater

# call the library function heater.py
simulate_heater(TAmb=200, stop_time=10)

## Compile the platform binary from source

In [ ]:
!fmpy compile Heater.fmu